# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(42)
initial_params = np.random.random([165])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Fetch Dataset

In [3]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

# Import dataset

In [4]:
examples = mnist.data
classes = mnist.target

x = []
y = []
for (example, label) in zip(examples, classes):
    if label in ["0", "1", "2", "3"]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=8)

In [9]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[1, 2])
    ArbitraryUnitary(params[15:30], wires=[3, 4])
    ArbitraryUnitary(params[30:45], wires=[5, 6])

    # Second layer
    ArbitraryUnitary(params[45:60], wires=[0, 1])
    ArbitraryUnitary(params[60:75], wires=[2, 3])
    ArbitraryUnitary(params[75:90], wires=[4, 5])
    ArbitraryUnitary(params[90:105], wires=[6, 7])

    # Third layer
    ArbitraryUnitary(params[105:120], wires=[2, 5])

    # Fourth layer
    ArbitraryUnitary(params[120:135], wires=[1, 2])
    ArbitraryUnitary(params[135:150], wires=[5, 6])

    # Fifth layer
    ArbitraryUnitary(params[150:165], wires=[2, 5])

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [10]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643
 0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184
 0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041
 0.60754485 0.17052412 0.06505159 0.94888554 0.96563203 0.80839735
 0.30461377 0.09767211 0.68423303 0.44015249 0.12203823 0.49517691
 0.03438852 0.9093204  0.25877998 0.66252228 0.31171108 0.52006802
 0.54671028 0.18485446 0.96958463 0.77513282 0.93949894 0.89482735
 0.59789998 0.92187424 0.0884925  0.19598286 0.04522729 0.32533033
 0.38867729 0.27134903 0.82873751 0.35675333 0.28093451 0.54269608
 0.14092422 0.80219698 0.07455064 0.98688694 0.77224477 0.19871568
 0.00552212 0.81546143 0.70685734 0.72900717 0.77127035 0.07404465
 0.35846573 0.11586906 0.86310343 0.62329813 0.33089802 0.06355835
 0.31098232 0.32518332 0.72960618 0.6375574

# Accuracy test definition

In [11]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [12]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.5980792316926771
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.7046818727490997
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.7001200480192077
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.724609843937575
best accuracy so far!
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.7157262905162065
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.7121248499399759
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.741656662665066
best accuracy so far!
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.7505402160864345
best accuracy so far!
epoch 71
epoch 72
epoch 73
epoc

epoch 348
epoch 349
epoch 350
accuracy: 0.7130852340936374
test_accuracies: [0.741656662665066, 0.7505402160864345, 0.7433373349339736, 0.7382953181272509, 0.741656662665066, 0.7267707082833132, 0.7104441776710684, 0.7166866746698679, 0.7186074429771909, 0.7361344537815127, 0.7042016806722688, 0.6864345738295319, 0.7339735894357743, 0.7250900360144057, 0.7445378151260504, 0.7601440576230492, 0.7294117647058824, 0.7262905162064826, 0.7505402160864345, 0.748859543817527, 0.7406962785114046, 0.7102040816326531, 0.7279711884753901, 0.7327731092436975, 0.7541416566626651, 0.7555822328931573, 0.7493397358943578, 0.7181272509003602, 0.7363745498199279, 0.7130852340936374]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7433373349339736
test_accuracies: [0.7505402160864345, 0.7433373349339736, 0.7382953181272509, 0.741656662665066, 0.7267707082833132, 0.7104441776710684, 0.7166866746698679, 0.7186074429771909, 0.7361344537815127, 

epoch 458
epoch 459
epoch 460
accuracy: 0.7558223289315726
test_accuracies: [0.6864345738295319, 0.7339735894357743, 0.7250900360144057, 0.7445378151260504, 0.7601440576230492, 0.7294117647058824, 0.7262905162064826, 0.7505402160864345, 0.748859543817527, 0.7406962785114046, 0.7102040816326531, 0.7279711884753901, 0.7327731092436975, 0.7541416566626651, 0.7555822328931573, 0.7493397358943578, 0.7181272509003602, 0.7363745498199279, 0.7130852340936374, 0.7433373349339736, 0.7418967587034814, 0.7433373349339736, 0.7363745498199279, 0.7284513805522208, 0.7162064825930372, 0.7406962785114046, 0.7248499399759905, 0.7495798319327731, 0.7469387755102042, 0.7558223289315726]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.7486194477791117
test_accuracies: [0.7339735894357743, 0.7250900360144057, 0.7445378151260504, 0.7601440576230492, 0.7294117647058824, 0.7262905162064826, 0.7505402160864345, 0.748859543817527, 0.7406962785114046

In [13]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [-0.2128753   1.03724574  0.56468742  0.4377067  -0.17590864  0.79254275
 -0.68236609  1.00555822  0.70613626 -0.1198024  -0.39756896  1.62716467
  0.71757527  0.01331699  0.9624403  -1.02542418 -0.07278717 -0.15515642
  0.98636892  0.92379417  1.20080738  0.10782553  0.64376949  0.15874912
  0.13640631  0.63280556 -0.28452896  0.73985948 -0.0956754  -0.66482301
  0.28090464  0.2976757  -0.43867074  1.00561911  1.53693036  0.91398405
 -0.44990007 -0.06499598  0.8774831   0.34466035  0.17972418 -0.34701569
  0.14390823 -0.23130801  0.41063723  0.85607272  0.70324853  0.11881492
  0.90579186 -0.16082561  0.25407564  0.65725752  1.63506937  1.37453753
  0.4001253   1.49107701  1.20629988 -0.26927863 -0.87951281  1.02609433
  0.6317557  -0.21222648  1.26252321  0.57203832 -0.04768072  0.61395488
  0.16562054  0.45799192  0.28246892  0.43523789  0.59696232  0.30311127
  0.02245736  0.53422087  0.70685734  1.12822668  1.52154958 -0.23447281
  0.28969881  0.38813724

# Testing

In [14]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7640952380952382


In [15]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 57.058434009552
training time: 55505.53121590614
test time: 2392.645541906357
total time: 57955.23519182205
